# Challenge: Walk Forward on Other Datasets

## Download data from `yfinance`

In [1]:
import yfinance as yf

ticker = 'AAPL'
df = yf.download(ticker)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099058,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086999,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089152,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091737,73449600
...,...,...,...,...,...,...
2024-06-03,192.899994,194.990005,192.520004,194.029999,194.029999,50080500
2024-06-04,194.639999,195.320007,193.029999,194.350006,194.350006,47471400
2024-06-05,195.399994,196.899994,194.869995,195.869995,195.869995,54156800


## Preprocess the data

### Filter the date range

In [2]:
df = df.loc['2018-01-01':].copy()

### Create the target variable

#### Percentage change

- Percentage change on `Adj Close` for tomorrow

In [3]:
df['change_tomorrow'] = df['Adj Close'].pct_change(-1)
df.change_tomorrow = df.change_tomorrow * -1
df.change_tomorrow = df.change_tomorrow * 100

#### Remove rows with any missing data

In [4]:
df = df.dropna().copy()
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow
Date,,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,40.615883,102223600,-0.017378
2018-01-03,43.132500,43.637501,42.990002,43.057499,40.608826,118071600,0.462327
2018-01-04,43.134998,43.367500,43.020000,43.257500,40.797443,89738400,1.125717
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.261936,94640000,-0.372815
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.108677,82271200,-0.011480
...,...,...,...,...,...,...,...
2024-05-31,191.440002,192.570007,189.910004,192.250000,192.250000,75158300,0.917383
2024-06-03,192.899994,194.990005,192.520004,194.029999,194.029999,50080500,0.164655
2024-06-04,194.639999,195.320007,193.029999,194.350006,194.350006,47471400,0.776019


## Machine Learning modelling

### Separate the data

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [5]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

### Time Series Split

In [6]:
from sklearn.model_selection import TimeSeriesSplit

ts = TimeSeriesSplit(test_size=200)

### Compute and evaluate model in a for loop

1. Separate the data in train and test
2. Compute the model on the train set
3. Evaluate the model (mse) on the test set
4. Append the errors (mse) in an empty list

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model_dt = RandomForestRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]
    
    model_dt.fit(X_train, y_train)
    
    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)
    
    error_mse_list.append(error_mse)

In [8]:
error_mse_list

[8.983950482811697,
 3.602141037449925,
 5.325989340029032,
 2.983740980302448,
 1.8130717722135927]

## Anchored Walk Forward evaluation in backtesting

![](<src/10_Table_Validation Methods.png>)

### Create a new strategy

In [9]:
from backtesting import Backtest, Strategy

In [10]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    n_train = 600
    coef_retrain = 200
    
    def init(self):
        self.model = RandomForestRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = self.data.df.iloc[:self.n_train, :-1]
        y_train = self.data.df.iloc[:self.n_train, -1]
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [11]:
class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()

### Run the backtest with optimization

In [12]:
from backtesting import Backtest
bt = Backtest(df, WalkForwardAnchored, cash=10000, commission=.002, exclusive_orders=True)

In [13]:
import multiprocessing as mp
mp.set_start_method('fork')

RuntimeError: context has already been set

In [ ]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

,limit_buy,limit_sell,Return [%]
0,0,-6,108.568730
1,0,-3,99.818273
...,...,...,...
2,0,-2,-22.922753
6,1,-1,-23.589944


## Unanchored Walk Forward

### Create a library of strategies

[strategies.py](strategies.py)

### Create the unanchored walk forward class

In the previously created library

![](<src/10_Table_Validation Methods.png>)

### Import the strategy and perform the backtest with optimization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import strategies

In [ ]:
strategies.WalkForwardUnanchored

strategies.WalkForwardUnanchored

In [ ]:
bt_unanchored = Backtest(df, strategies.WalkForwardUnanchored, cash=10000, commission=.002, exclusive_orders=True)

stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

,limit_buy,limit_sell,Return [%]
0,0,-6,106.324051
4,1,-6,106.324051
5,1,-5,83.998857
1,0,-5,76.952137
9,2,-6,59.677444
23,5,-5,18.879328
18,4,-6,18.067427
15,3,-4,13.795739
6,1,-4,13.435896
14,3,-5,11.242048


### Interpret the strategies' performance

In [ ]:
bt.plot(filename='reports_backtesting/walk_forward_anchored.html')

In [ ]:
bt_unanchored.plot(filename='reports_backtesting/walk_forward_unanchored.html')

## Course Conclusion

Watch video → [Next steps]()